### Zastosowanie wymaga dyskretyzacji action space

In [1]:
import gym
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
from math import radians, inf

In [2]:
# %load model.py
from math import sin, cos
import numpy as np
from model_parameters import m, g, I, l, Kt, Kd

def transfomationMatrix(roll, pitch, yaw):
    cR = cos(roll)
    cP = cos(pitch)
    cY = cos(yaw)
    sR = sin(roll)
    sP = sin(pitch)
    sY = sin(yaw)
    R = np.array([[cP*cY, sR*sP*cY - cR*sY, cR*sP*cY + sR*sY],
                  [cP*sY, sR*sP*sY - cR*cY, cR*sP*sY - sR*cY],
                  [-sP, sR*cP, cR*cP]], dtype=np.float32)
    return R

def translationalMotion(R, F):
    f = F[0] + F[1]  + F[2] + F[3]
    accelerations = np.zeros((3, 1), dtype=np.float32)
    bodyFrameThrust = np.array([[0], 
                               [0],
                               [f]], dtype=np.float32)
    referenceFrame = np.matmul(R, bodyFrameThrust) + np.array([[0],
                                                               [0],
                                                               [-m*g]], dtype=np.float32)
    accelerations = referenceFrame/m
    accelerations = np.reshape(accelerations, 3)
    return accelerations

def angularMotion(F, M, eulerAnglesPrim):
    T = np.zeros(3)
    T[0] = (F[0] - F[3])*l
    T[1] = (F[1] - F[2])*l
    T[2] = M[0] + M[1] + M[2] + M[3]
    accelerations =  T - np.cross(eulerAnglesPrim, np.multiply(I, eulerAnglesPrim))
    accelerations = np.divide(accelerations, I)
    return accelerations

def inputToForces(omega):
    return np.multiply(Kt, omega)

def inputToMomentum(omega):
    return np.multiply(Kd, omega)

def model(x):
    dstate = np.zeros(12)
    omega = np.array([2000, 2000, 2000, 2000])
    F = inputToForces(omega)
    M = inputToMomentum(omega)
    R = transfomationMatrix(x[3], x[4], x[5])
    dstate[0:3] = x[3:6]
    dstate[3:6] = translationalMotion(R, F)
    dstate[6:9] = x[9:12]
    dstate[9:12] = angularMotion(F, M, x[9:12])
    return dstate 

def modelRT(x, u, deltaT):
    state = np.zeros(12)
    omega = np.array(u)
    F = inputToForces(omega)
    M = inputToMomentum(omega)
    R = transfomationMatrix(x[3], x[4], x[5])
    state[0:3] = x[3:6] * deltaT + x[0:3]
    state[3:6] = translationalMotion(R, F) * deltaT + x[3:6]
    state[6:9] = x[9:12] * deltaT + x[6:9]
    state[9:12] = angularMotion(F, M, x[9:12]) * deltaT + x[9:12]
    return state 

In [3]:
class DroneEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=-inf, high=inf, shape=(1, 12))
        self.state = np.zeros(12)
        self.state[2] = 5
        self.maximum_angle = radians(30)
        self.flight_length = 10
        self.deltaT = 0.1
        self.u = np.array([0, 0, 0, 0])
        
    def step(self, action):
        if action == 1:
            self.u = self.u + 500
        elif action == 2:
            self.u = self.u - 500
        self.state = modelRT(self.state, self.u, self.deltaT)
        self.flight_lenght = self.flight_length-self.deltaT
        reward = - abs(5 - self.state[2])
        print(self.state[2])
        if self.state[2] < 0 or self.flight_length < 0:
            done = True
        else:
            done = False
        info = {}
        return self.state, reward, done, info
        
    def render(self):
        #visualization
        pass
    def reset(self):
        self.state = np.zeros(12)
        self.state[2] = 5
        self.flight_lenght = 10
        return self.state

In [4]:
env = DroneEnv()

In [5]:
env.action_space.sample()

2

In [6]:
# episodes = 20
# for episode in range(1, episodes+1):
#     state = env.reset()
#     done = False
#     score = 0

#     while not done:
#         #env.render()
#         action = env.action_space.sample()
#         n_state, reward, done, info = env.step(action)
#         score+=reward
#     print('Episode:{} Score:{}'. format(episode, score))

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [8]:
states = env.observation_space.shape
actions = 3

In [9]:
def build_model(states, actions):
    model = Sequential()
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    model.add(Flatten())
    return model

In [10]:
del CNNmodel

NameError: name 'CNNmodel' is not defined

In [ ]:
CNNmodel = build_model(states, actions)

In [ ]:
CNNmodel.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 1, 24)             312       
                                                                 
 dense_4 (Dense)             (None, 1, 24)             600       
                                                                 
 dense_5 (Dense)             (None, 1, 3)              75        
                                                                 
 flatten_1 (Flatten)         (None, 3)                 0         
                                                                 
Total params: 987
Trainable params: 987
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy 
from rl.memory import SequentialMemory

In [ ]:
def buildAgent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(CNNmodel, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10,
                   target_model_update=1e-2)
    return dqn

In [ ]:
dqn = buildAgent(CNNmodel, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

2022-08-01 16:20:43.726716: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Training for 50000 steps ...
Interval 1 (0 steps performed)
5.0
    1/10000 [..............................] - ETA: 12:21 - reward: 0.0000e+004.9019609034061435
4.705821800231934
4.411521780490876
4.019121754169465
3.528560811281205
2.939778023958207
2.2527733922004707
1.4675469160079961
0.5840985953807836
-0.39757156968116714


/home/pete/anaconda3/envs/AdaptiveDrone/lib/python3.8/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
/home/pete/anaconda3/envs/AdaptiveDrone/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


5.0
   12/10000 [..............................] - ETA: 6:46 - reward: -1.7989    4.901778155565262
4.705273550748825
4.410425275564194
4.017172402143478
3.5254540205001828
2.9352701306343074
2.2465598165988916
1.4592621684074394
0.5733771860599509
-0.4111560583114633
5.0
   23/10000 [..............................] - ETA: 3:50 - reward: -1.87934.901351737976074
4.704116141796112
4.408232283592223
4.013639253377914
3.5202761352062217
2.928203845024108
2.237361466884612
1.4476880729198442
0.5591227531433092
-0.42833449244499344
   33/10000 [..............................] - ETA: 2:54 - reward: -1.96765.0
4.900986230373382
4.702897781133651
4.405795562267303
4.009618663787841
3.5144279956817623
2.9202235579490656
2.2269444406032557
   41/10000 [..............................] - ETA: 2:32 - reward: -1.78661.4345297336578362
0.5429185032844536
-0.4479501605033883
5.0
4.900681656599045
4.70210587978363
4.404211759567261
4.00693838596344
   49/10000 [..............................] - ETA: 2: